# Data Science: Microeconomic panel data

In [1]:
import warnings
import numpy as np
import pandas as pd
from cleands import *
from itertools import product

class panelDF(pd.DataFrame):
    def __init__(self,*args,**kwargs):
        id_dim = kwargs.pop('id',None)
        time_dim = kwargs.pop('time',None)
        super(panelDF,self).__init__(*args,**kwargs)    
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self.id = id_dim
            self.time = time_dim
        self.sort_values(by=[id_dim,time_dim],inplace=True)
    def outer_balance(self):
        for j in product(self[self.id].unique(),self[self.time].unique()):
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                rows = self[self[self.id]==j[0]][self[self.time]==j[1]]
            if len(rows)<1:
                data = {i:np.nan for i in self.columns}            
                data[self.id] = j[0]
                data[self.time] = j[1]
                row = pd.DataFrame(data,index=[0])
                df = pd.concat((self,row),axis=0,ignore_index=True)
                self = panelDF(df,id=self.id,time=self.time)
            elif len(rows)>1:
                raise Exception('duplicate rows')
        return self


In [4]:
df = pd.read_csv('JTRAIN.csv')
df = df[['year','fcode','scrap','grant','grant_1']]
df['lscrap'] = np.log(df['scrap'])
df = df.dropna()
#df = df[df.index!=30]
df = panelDF(df,id='fcode',time='year')
df = df.outer_balance()
df

,year,fcode,scrap,grant,grant_1,lscrap
30,1987,410523,0.06,0,0,-2.813411
31,1988,410523,0.05,0,0,-2.995732
32,1989,410523,0.05,0,0,-2.995732
48,1987,410538,2.64,0,0,0.970779
49,1988,410538,2.74,0,0,1.007958
...,...,...,...,...,...,...
463,1988,419482,2.59,0,0,0.951658
464,1989,419482,2.80,0,0,1.029619
465,1987,419483,20.00,0,0,2.995732
466,1988,419483,25.00,0,0,3.218876


In [5]:
162/3

54.0

In [6]:
# Pooled OLS
LeastSquaresRegressor(*add_intercept(['grant','grant_1'],'lscrap',df)).tidy

,variable,estimate,std.error,t.statistic,p.value
0,(intercept),0.41505637487690583,0.13982796899679673,2.9683358619505817,0.0034576102640516255
1,grant,0.05435340639144301,0.31050104728953193,0.17505063788322833,0.8612625593662346
2,grant_1,-0.26521024220789774,0.3699500322967739,-0.7168812516689986,0.4744991094744584


In [15]:
ddf = df.copy()
means_t = ddf[df.columns].groupby(df.time).mean().reset_index()[df.columns]
np.kron(np.ones((len(df[df.id].unique()),1)),means_t).shape

(162, 6)

In [12]:
np.kron(np.ones((3,1)),means_t)

array([[1.98700000e+03, 4.16313574e+05, 4.61166667e+00, 0.00000000e+00,
        0.00000000e+00, 5.97434051e-01],
       [1.98800000e+03, 4.16313574e+05, 3.78777778e+00, 3.51851852e-01,
        0.00000000e+00, 4.28440922e-01],
       [1.98900000e+03, 4.16313574e+05, 3.13148148e+00, 1.85185185e-01,
        3.51851852e-01, 1.55169229e-01],
       [1.98700000e+03, 4.16313574e+05, 4.61166667e+00, 0.00000000e+00,
        0.00000000e+00, 5.97434051e-01],
       [1.98800000e+03, 4.16313574e+05, 3.78777778e+00, 3.51851852e-01,
        0.00000000e+00, 4.28440922e-01],
       [1.98900000e+03, 4.16313574e+05, 3.13148148e+00, 1.85185185e-01,
        3.51851852e-01, 1.55169229e-01],
       [1.98700000e+03, 4.16313574e+05, 4.61166667e+00, 0.00000000e+00,
        0.00000000e+00, 5.97434051e-01],
       [1.98800000e+03, 4.16313574e+05, 3.78777778e+00, 3.51851852e-01,
        0.00000000e+00, 4.28440922e-01],
       [1.98900000e+03, 4.16313574e+05, 3.13148148e+00, 1.85185185e-01,
        3.51851852e-01, 

In [16]:
ddf = df.copy()
means_t = ddf[df.columns].groupby(df.time).mean().reset_index()[df.columns]
ddf -= np.kron(np.ones((len(df[df.id].unique()),1)),means_t)

# Common time effects
model = LeastSquaresRegressor(['grant','grant_1'],'lscrap',ddf)
model.tidy

,variable,estimate,std.error,t.statistic,p.value
0,grant,0.20001966562375645,0.335098178670527,0.5968986952340868,0.5514187067373635
1,grant_1,0.048935698577833,0.431958857393869,0.11328786929634001,0.909944354348349


In [17]:
time_fe = means_t[model.y_var]-means_t[model.x_vars]@model.params
time_fe

0    0.597434
1    0.358064
2    0.100910
dtype: float64

In [18]:
means_t

,year,fcode,scrap,grant,grant_1,lscrap
0,1987,416313.574074,4.611667,0.000000,0.000000,0.597434
1,1988,416313.574074,3.787778,0.351852,0.000000,0.428441
2,1989,416313.574074,3.131481,0.185185,0.351852,0.155169


In [19]:
np.kron(means_i,np.ones((T,1)))

array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [3., 4.],
       [3., 4.],
       [3., 4.]])

In [20]:
ddf = df.copy()
means_i = ddf[df.columns].groupby(df.id).mean().reset_index()[df.columns]
ddf -= np.kron(means_i,np.ones((len(df[df.time].unique()),1)))

In [21]:
ddf = df.copy()
means_i = ddf[df.columns].groupby(df.id).mean().reset_index()[df.columns]
ddf -= np.kron(means_i,np.ones((len(df[df.time].unique()),1)))
means_t = ddf[df.columns].groupby(df.time).mean().reset_index()[df.columns]
ddf -= np.kron(np.ones((len(df[df.id].unique()),1)),means_t)

# Two way effects
model = LeastSquaresRegressor(['grant','grant_1'],'lscrap',ddf)
model.tidy

,variable,estimate,std.error,t.statistic,p.value
0,grant,-0.2523148711500044,0.12144097568600476,-2.077674934055079,0.0393367004795288
1,grant_1,-0.4215895094264221,0.16946862639252946,-2.4877142064627407,0.013881528983727506


In [22]:
n = len(df[df.id].unique())
T = len(df[df.time].unique())
x = model.x.values
r = x.shape[1]
e = model.residuals.values.reshape(-1,1)
vc = np.zeros((r,r))
for i in range(n):
    xi = x[i*T:(i+1)*T,:]
    ei = e[i*T:(i+1)*T,:]
    vc += xi.T@ei@ei.T@xi
ixx = np.linalg.inv(x.T@x)
vc = ixx@vc@ixx
vc # arellano 1987 HAC estimator

array([[0.01969226, 0.03448178],
       [0.03448178, 0.07636089]])

In [29]:
15*16/2

120.0

In [23]:
se = np.sqrt(np.diag(vc))
se

array([0.14032911, 0.27633474])

In [20]:
model.tidy

,variable,estimate,std.error,t.statistic,p.value
0,grant,-0.2523148711500044,0.12144097568600476,-2.077674934055079,0.0393367004795288
1,grant_1,-0.4215895094264221,0.16946862639252946,-2.4877142064627407,0.013881528983727506


In [24]:
-0.2523148711500044/0.14032911

-1.7980223144720606

In [25]:
indiv_fe = means_i[model.y_var]-means_i[model.x_vars]@model.params
time_fe = means_t[model.y_var]-means_t[model.x_vars]@model.params
time_fe

0    0.109139
1    0.028924
2   -0.138063
dtype: float64

In [26]:
indiv_fe

0    -2.934958
1     0.970300
2     1.782369
3     1.508709
4     1.686475
5    -0.655371
6     0.488189
7     3.191661
8     0.000000
9     1.826880
10   -0.811978
11   -0.226734
12   -0.388636
13    0.072952
14   -3.009340
15   -1.756123
16    1.589416
17    0.224635
18    1.724571
19    0.693147
20   -0.503633
21    0.429336
22    0.447245
23   -0.335033
24    0.821888
25    0.598433
26    0.564796
27   -0.272717
28   -0.773943
29   -0.376575
30    1.685310
31    2.229614
32   -2.807294
33    3.133029
34   -1.289743
35    1.871849
36    0.693147
37    1.712000
38    0.208226
39    3.094669
40    0.316692
41   -1.363318
42   -0.008913
43   -0.673096
44    1.448886
45    0.084105
46    0.653548
47   -0.581296
48    2.173804
49    0.890894
50    1.608904
51    0.514148
52    0.991510
53    3.205268
dtype: float64

In [27]:
%pip install --user linearmodels

Note: you may need to restart the kernel to use updated packages.


In [25]:
df = df.set_index(['fcode','year'])

In [26]:
from linearmodels import PanelOLS
mod = PanelOLS(df.lscrap, df[['grant','grant_1']], entity_effects=True, time_effects=True)
res = mod.fit(cov_type='kernel')
res

Dep. Variable:,lscrap,R-squared:,0.0411
Estimator:,PanelOLS,R-squared (Between):,-0.0552
No. Observations:,162,R-squared (Within):,0.1478
Date:,"Wed, Nov 16 2022",R-squared (Overall):,-0.0381
Time:,13:11:07,Log-likelihood,-80.946
Cov. Estimator:,Driscoll-Kraay,,
,,F-statistic:,2.2294
Entities:,54,P-value,0.1127
Avg Obs:,3.0000,Distribution:,"F(2,104)"
Min Obs:,3.0000,,
Max Obs:,3.0000,F-statistic (robust):,92.755


# Data Science: Macroeconomic panel data

In [30]:
df = pd.read_csv('PRISON.csv')
gb = df.groupby(['year'])['criv'].apply(list).apply(pd.Series).rename(columns=lambda x: 'State_{0}'.format(x+1))
gb

,State_1,State_2,State_3,State_4,State_5,State_6,State_7,State_8,State_9,State_10,...,State_42,State_43,State_44,State_45,State_46,State_47,State_48,State_49,State_50,State_51
year,,,,,,,,,,,,,,,,,,,,,
80,4.447868,4.773632,6.502207,3.349081,8.884993,5.264706,4.108108,4.754209,20.75549,9.654730,...,1.263386,4.535831,5.480217,3.028747,1.788650,3.058724,4.621975,1.818974,1.815980,3.917021
81,4.700944,6.069378,5.723132,3.104666,8.584576,5.290463,4.484819,5.107383,23.61852,9.625134,...,1.040580,4.109550,5.327547,2.988119,1.277132,3.206466,4.447356,1.745138,1.884046,4.300813
82,4.497580,6.071111,5.115917,3.242371,8.115592,5.014369,4.013062,5.626043,21.91640,8.920924,...,0.989870,4.215669,5.751615,2.849808,1.262042,3.089386,4.033435,1.746154,1.919433,3.017787
83,4.186833,6.024590,4.931627,3.005204,7.669203,4.771857,3.721379,4.538843,19.57120,8.213209,...,1.212121,4.042060,5.113382,2.598119,1.330784,2.917520,3.717674,1.735219,1.921203,2.390196
84,4.353239,6.046692,5.137268,3.253879,7.568062,4.589274,3.906289,4.364379,17.71406,8.629529,...,1.493544,4.482612,5.043856,2.487669,1.459203,2.951807,4.069061,1.702801,1.977829,2.449505
85,4.630758,5.697369,6.030779,3.523421,7.630687,4.742599,3.985630,4.360841,16.02992,9.423927,...,1.385387,4.791516,5.532477,2.676811,1.490566,2.941907,4.262955,1.685370,2.080876,2.614000
86,5.665331,5.599265,6.600665,4.015866,9.164268,5.284214,4.211538,4.304140,14.76959,10.371360,...,1.268678,5.469297,6.637582,2.670475,1.511236,3.046800,4.380193,1.676939,2.594407,2.995968
87,5.686924,4.435993,6.034914,4.201110,9.142204,4.726687,4.143825,4.354788,15.72371,10.266570,...,1.219828,5.415011,6.374744,2.300954,1.385185,2.935604,4.400927,1.402045,2.514441,2.907757
88,5.728628,4.948339,5.982178,4.369185,9.201518,4.766401,4.510697,4.600308,18.91111,11.241910,...,1.164756,5.434467,6.569808,2.433393,1.438182,2.971675,4.642888,1.353005,2.159685,3.180645


In [28]:
df = pd.read_csv('PRISON.csv')
gb = df.groupby(['year'])['criv'].apply(list).apply(pd.Series).rename(columns=lambda x: 'State_{0}'.format(x+1))
gb

,State_1,State_2,State_3,State_4,State_5,State_6,State_7,State_8,State_9,State_10,...,State_42,State_43,State_44,State_45,State_46,State_47,State_48,State_49,State_50,State_51
year,,,,,,,,,,,,,,,,,,,,,
80,4.447868,4.773632,6.502207,3.349081,8.884993,5.264706,4.108108,4.754209,20.75549,9.654730,...,1.263386,4.535831,5.480217,3.028747,1.788650,3.058724,4.621975,1.818974,1.815980,3.917021
81,4.700944,6.069378,5.723132,3.104666,8.584576,5.290463,4.484819,5.107383,23.61852,9.625134,...,1.040580,4.109550,5.327547,2.988119,1.277132,3.206466,4.447356,1.745138,1.884046,4.300813
82,4.497580,6.071111,5.115917,3.242371,8.115592,5.014369,4.013062,5.626043,21.91640,8.920924,...,0.989870,4.215669,5.751615,2.849808,1.262042,3.089386,4.033435,1.746154,1.919433,3.017787
83,4.186833,6.024590,4.931627,3.005204,7.669203,4.771857,3.721379,4.538843,19.57120,8.213209,...,1.212121,4.042060,5.113382,2.598119,1.330784,2.917520,3.717674,1.735219,1.921203,2.390196
84,4.353239,6.046692,5.137268,3.253879,7.568062,4.589274,3.906289,4.364379,17.71406,8.629529,...,1.493544,4.482612,5.043856,2.487669,1.459203,2.951807,4.069061,1.702801,1.977829,2.449505
85,4.630758,5.697369,6.030779,3.523421,7.630687,4.742599,3.985630,4.360841,16.02992,9.423927,...,1.385387,4.791516,5.532477,2.676811,1.490566,2.941907,4.262955,1.685370,2.080876,2.614000
86,5.665331,5.599265,6.600665,4.015866,9.164268,5.284214,4.211538,4.304140,14.76959,10.371360,...,1.268678,5.469297,6.637582,2.670475,1.511236,3.046800,4.380193,1.676939,2.594407,2.995968
87,5.686924,4.435993,6.034914,4.201110,9.142204,4.726687,4.143825,4.354788,15.72371,10.266570,...,1.219828,5.415011,6.374744,2.300954,1.385185,2.935604,4.400927,1.402045,2.514441,2.907757
88,5.728628,4.948339,5.982178,4.369185,9.201518,4.766401,4.510697,4.600308,18.91111,11.241910,...,1.164756,5.434467,6.569808,2.433393,1.438182,2.971675,4.642888,1.353005,2.159685,3.180645


In [31]:
def kpss(series,trend=False,lagshort=True):
    y = series.values if type(series)==pd.Series else series
    n = y.shape[0]
    if lagshort: 
        lags = int(4*(n/100)**0.25)
    else: 
        lags = int(12*(n/100)**0.25)
    y = y.reshape(-1,1)
    x = np.ones((n,1))
    if trend:
        x = np.hstack((x,np.arange(n).reshape(-1,1)))
        table = np.array([0.119,0.146,0.176,0.216])
    else:
        table = np.array([0.347,0.463,0.574,0.739])
    tablep = np.array([0.1,0.05,0.025,0.01])
    res = y-x@np.linalg.solve(x.T@x,x.T@y)
    s = res.cumsum(0)
    eta = s.T@s/n**2
    sig = [2*(1-i/(lags+1))*(res[i:,:].T@res[:res.shape[0]-i,:])[0,0]/res[i:,:].shape[0] for i in range(1,lags+1)]
    sig = np.array(sig).sum()
    sig += (res.T@res)[0,0]/res.shape[0]
    stat = eta/sig
    p = np.interp(stat,table,tablep)
    df = pd.DataFrame(columns=['method','stat','lags','p'])
    df.loc[0] = ['kpss',stat[0,0],lags,p[0,0]]
    return df

In [33]:
kpss(gb.mean(1).diff().dropna())

,method,stat,lags,p
0,kpss,0.297749,2,0.1


In [37]:
for i in range(51):
    print(kpss(gb.values[:,i]))

  method      stat  lags         p
0   kpss  0.476136     2  0.047041
  method      stat  lags    p
0   kpss  0.151532     2  0.1
  method      stat  lags         p
0   kpss  0.350579     2  0.098457
  method      stat  lags         p
0   kpss  0.498951     2  0.041903
  method      stat  lags         p
0   kpss  0.426743     2  0.065628
  method      stat  lags    p
0   kpss  0.247562     2  0.1
  method      stat  lags        p
0   kpss  0.350388     2  0.09854
  method      stat  lags    p
0   kpss  0.337382     2  0.1
  method      stat  lags    p
0   kpss  0.253696     2  0.1
  method      stat  lags         p
0   kpss  0.449728     2  0.055721
  method      stat  lags         p
0   kpss  0.435551     2  0.061831
  method     stat  lags    p
0   kpss  0.10984     2  0.1
  method      stat  lags    p
0   kpss  0.149069     2  0.1
  method      stat  lags         p
0   kpss  0.503912     2  0.040786
  method      stat  lags         p
0   kpss  0.405482     2  0.074792
  method      

In [38]:
for i in range(51):
    print(kpss(gb.values[:-1,i]-gb.values[1:,i]))

  method      stat  lags    p
0   kpss  0.130792     2  0.1
  method      stat  lags    p
0   kpss  0.185776     2  0.1
  method      stat  lags    p
0   kpss  0.228057     2  0.1
  method      stat  lags    p
0   kpss  0.250979     2  0.1
  method      stat  lags    p
0   kpss  0.218162     2  0.1
  method      stat  lags    p
0   kpss  0.230104     2  0.1
  method      stat  lags    p
0   kpss  0.117884     2  0.1
  method      stat  lags    p
0   kpss  0.168877     2  0.1
  method      stat  lags    p
0   kpss  0.266638     2  0.1
  method      stat  lags    p
0   kpss  0.153876     2  0.1
  method      stat  lags    p
0   kpss  0.150953     2  0.1
  method      stat  lags    p
0   kpss  0.285337     2  0.1
  method      stat  lags         p
0   kpss  0.351829     2  0.097918
  method      stat  lags    p
0   kpss  0.191134     2  0.1
  method      stat  lags    p
0   kpss  0.268172     2  0.1
  method     stat  lags    p
0   kpss  0.24624     2  0.1
  method      stat  lags    p
0 

In [39]:
df = df[['state','year','criv','polpc']]
df = df.dropna()
df = panelDF(df,id='state',time='year')
df = df.outer_balance()
df['lcriv'] = np.log(df['criv'])
df['gcriv'] = df['lcriv'].diff()
df.loc[df['year']==80,'gcriv'] = np.nan
df['lpolpc'] = np.log(df['polpc'])
df['gpolpc'] = df['lpolpc'].diff()
df.loc[df['year']==80,'gpolpc'] = np.nan
df = df.dropna()
df = panelDF(df,id='state',time='year')


In [40]:
ddf = df.copy()
means_i = ddf[df.columns].groupby(df.id).mean().reset_index()[df.columns]
ddf -= np.kron(means_i,np.ones((len(df[df.time].unique()),1)))
means_t = ddf[df.columns].groupby(df.time).mean().reset_index()[df.columns]
ddf -= np.kron(np.ones((len(df[df.id].unique()),1)),means_t)

LeastSquaresRegressor(['lpolpc'],'lcriv',data=ddf).tidy

,variable,estimate,std.error,t.statistic,p.value
0,lpolpc,0.38493179181275955,0.06855469454216134,5.614958893530265,2.8946908042274615e-08


In [41]:
LeastSquaresRegressor(['gpolpc'],'gcriv',data=ddf).tidy

,variable,estimate,std.error,t.statistic,p.value
0,gpolpc,0.044772693256053146,0.05779963803479563,0.7746189211271495,0.4388414459491329


In [42]:
from linearmodels import PanelOLS
import statsmodels.api as sm
data = df.set_index(['state','year'])
dependent = data.gcriv
exog = sm.add_constant(data[['gpolpc']])
mod = PanelOLS(dependent, exog, entity_effects=True, time_effects=True)
res = mod.fit(cov_type='kernel')
res

Dep. Variable:,gcriv,R-squared:,0.0009
Estimator:,PanelOLS,R-squared (Between):,0.0126
No. Observations:,663,R-squared (Within):,0.0022
Date:,"Wed, Nov 16 2022",R-squared (Overall):,0.0028
Time:,19:27:16,Log-likelihood,760.61
Cov. Estimator:,Driscoll-Kraay,,
,,F-statistic:,0.5429
Entities:,51,P-value,0.4615
Avg Obs:,13.000,Distribution:,"F(1,599)"
Min Obs:,13.000,,
Max Obs:,13.000,F-statistic (robust):,0.2902


# Programming challenge

## Panel Bootstrap

Write a panel data bootstrap for the PRISON.csv data
